# DATA

In [1]:
import numpy as np
import pandas as pd

In [2]:
vendas = pd.read_csv('../../../data/interim/Vendas.csv', encoding='ISO-8859-1')

In [3]:
vendas = vendas.replace(['ST', 'CORSÁRIO SUPLEX', 'ESP', 'ESPECIAL'], ['00','CORSARIO SUPLEX', 'GG', 'GG'])
vendas.head()

,ANO,DESCRICAO,DIA,HORA,MES,NOME,QUANTIDADE,TAMANHO,VLR_UNIT
0,2016.0,CAMISETA MANGA CURTA,12.0,14:54,1.0,VERBO DIVINO,7.0,00,35.5
1,2016.0,BERMUDA CICLISTA COTTON,12.0,15:08,1.0,VERBO DIVINO,1.0,08,51.0
2,2016.0,CAMISETA MANGA CURTA,12.0,15:08,1.0,VERBO DIVINO,5.0,PP,37.5
3,2016.0,BLUSA MOLETOM FLANELADA,12.0,15:27,1.0,VERBO DIVINO,1.0,M,115.0
4,2016.0,CALÇA SARJA MASC,12.0,15:27,1.0,VERBO DIVINO,1.0,GG,116.0


In [4]:
a = vendas.groupby(['ANO', 'MES', 'DESCRICAO', 'NOME', 'TAMANHO'])

In [5]:
df = a.sum().reset_index().drop(['DIA', 'VLR_UNIT'], 1)
df = df.sort_values(by=['ANO','MES', 'DESCRICAO', 'TAMANHO', 'NOME'])

In [6]:
df_vlr_unit = vendas.groupby(['ANO', 'MES', 'DESCRICAO', 'NOME', 'TAMANHO']).mean()
df = df.merge(df_vlr_unit.drop(['QUANTIDADE', 'DIA'], axis=1), on=['ANO', 'MES', 'DESCRICAO', 'NOME', 'TAMANHO'], how='left')

In [7]:
cols = df.columns.tolist()
y = cols.index('QUANTIDADE')
cols = cols[:y] + cols[y+1:] + ['QUANTIDADE']
df = df[cols]

In [8]:
df.head()

,ANO,MES,DESCRICAO,NOME,TAMANHO,VLR_UNIT,QUANTIDADE
0,2016.0,1.0,BERMUDA CICLISTA COLEGIAL,VERBO DIVINO,M,57.5,1.0
1,2016.0,1.0,BERMUDA CICLISTA COLEGIAL,ALDEIA DA SERRA,P,57.5,1.0
2,2016.0,1.0,BERMUDA CICLISTA COLEGIAL,VERBO DIVINO,P,57.5,1.0
3,2016.0,1.0,BERMUDA CICLISTA COLEGIAL,ALDEIA DA SERRA,PP,57.5,1.0
4,2016.0,1.0,BERMUDA CICLISTA COLEGIAL,ITAIM,PP,57.5,1.0


In [9]:
df.head()

,ANO,MES,DESCRICAO,NOME,TAMANHO,VLR_UNIT,QUANTIDADE
0,2016.0,1.0,BERMUDA CICLISTA COLEGIAL,VERBO DIVINO,M,57.5,1.0
1,2016.0,1.0,BERMUDA CICLISTA COLEGIAL,ALDEIA DA SERRA,P,57.5,1.0
2,2016.0,1.0,BERMUDA CICLISTA COLEGIAL,VERBO DIVINO,P,57.5,1.0
3,2016.0,1.0,BERMUDA CICLISTA COLEGIAL,ALDEIA DA SERRA,PP,57.5,1.0
4,2016.0,1.0,BERMUDA CICLISTA COLEGIAL,ITAIM,PP,57.5,1.0


In [10]:
treino.shape, validacao.shape

NameError: name 'treino' is not defined

In [11]:
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, StandardScaler

In [12]:
le_DESC = LabelEncoder()
# treino['DESCRICAO'] = le_DESC.fit_transform(treino.DESCRICAO)
# validacao['DESCRICAO'] = le_DESC.transform(validacao.DESCRICAO)
df['DESCRICAO'] = le_DESC.fit_transform(df.DESCRICAO)

In [13]:
tamanho_val = {
    '00' : 0,
    '02' : 1,
    '04' : 2,
    '06' : 3,
    '08' : 4,
    '10' : 5,
    '12' : 6,
    '14' : 7,
    'PP' : 8,
    'P' : 9,
    'M' : 10,
    'G' : 11,
    'GG' : 12
}
# treino.TAMANHO = treino.TAMANHO.apply(lambda x: tamanho_val[x])
# validacao.TAMANHO = validacao.TAMANHO.apply(lambda x: tamanho_val[x])
df.TAMANHO = df.TAMANHO.apply(lambda x: tamanho_val[x])

In [14]:
le_NOME = LabelEncoder()
# treino['NOME'] = le_NOME.fit_transform(treino.NOME)
# validacao['NOME'] = le_NOME.transform(validacao.NOME)
df['NOME'] = le_NOME.fit_transform(df.NOME)

In [18]:
sc = StandardScaler()
df = sc.fit_transform(df)

In [20]:
df

array([[-1.02020802, -1.02077594, -1.80068256, ...,  1.45792144,
         0.        , -0.42347879],
       [-1.02020802, -1.02077594, -1.80068256, ...,  1.15511631,
         0.        , -0.42347879],
       [-1.02020802, -1.02077594, -1.80068256, ...,  1.15511631,
         0.        , -0.42347879],
       ...,
       [ 1.67612761,  0.23759642,  1.79425871, ..., -0.66171446,
         0.        , -0.25506377],
       [ 1.67612761,  0.23759642,  1.79425871, ..., -0.35890933,
         0.        , -0.08664876],
       [ 1.67612761,  0.23759642,  1.79425871, ..., -0.0561042 ,
         0.        , -0.33927128]])

# FEATURES

In [40]:
df.columns

Index(['ANO', 'MES', 'DESCRICAO', 'NOME', 'TAMANHO', 'QUANTIDADE'], dtype='object')

# TRAIN

In [31]:
treino = df[df.ANO < 2018]
validacao = df[df.ANO >= 2018]
y_train = treino['QUANTIDADE']
X_train = treino.drop('QUANTIDADE', 1)
X_test = validacao.drop('QUANTIDADE', 1)
Y_test = validacao['QUANTIDADE']

In [37]:
from sklearn.ensemble import RandomForestRegressor, ExtraTreesRegressor, GradientBoostingRegressor 
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import mean_squared_error
from xgboost import XGBRegressor
from math import sqrt

In [38]:
models = []
models.append(('LR', LogisticRegression()))
models.append(('RF', RandomForestRegressor()))
models.append(('XTREE', ExtraTreesRegressor()))
models.append(('GBR', GradientBoostingRegressor()))
models.append(('XGB', XGBRegressor()))
results = []
for name, model in models:
    model.fit(X_train, y_train)
    Y_pred = model.predict(X_test)
    print(name, sqrt(mean_squared_error(Y_test, Y_pred)))

LR 15.05185621890263
RF 6.588094168389945
XTREE 7.943733602865814
GBR 9.097705257913706
XGB 9.062065924958528


# RANDOM FOREST

# TESTES

In [28]:
vendas.QUANTIDADE.unique()

array([ 7.,  1.,  5.,  6.,  2.,  3.,  4., 12.,  8.,  9., 10.])